## librerias y funciones extra

In [1]:
!pip install kora -q
from kora import drive
drive.link_nbs()

In [2]:
import numpy as np
import scipy.spatial
import time
import pandas as pd
from sklearn.metrics import davies_bouldin_score as DB
from sklearn.metrics import confusion_matrix as confusion
from sklearn.metrics import completeness_score as completitud
from sklearn.metrics import adjusted_rand_score as ari
import pickle

In [3]:
from ACDE_EvolucionDiferencial import *

importing Jupyter notebook from /nbs/ACDE_EvolucionDiferencial.ipynb


In [4]:
def D_intra_inter(particion,activos,datos):
  '''obtenemos la distancia intra cluster de los grupos y la distancia
  entre clusters para comparar contra el articulo original
  NO USAR PARA COMPARAR METAHEURISTICAS ENTRE SI'''
  elementos_g=[]
  for i in range(len(activos)):
    elementos_g.append(np.where(particion == i)[0].tolist())
    activos[i]=np.mean(datos[elementos_g[i]],axis=0)
  dividendo = 0
  divisor = 0
  #iteramos por grupos para sacar su distancia interna y entre grupos
  for i,centroide in enumerate(activos):
    intra = 0; elementos = elementos_g[i]; nPuntosG = len(elementos)
    #sacamos la distancia interna del grupo
    for j,elemento in enumerate(elementos):
      vecinos = elementos[0:j] + elementos[j+1::]
      intra+= max([np.linalg.norm(datos[elemento]-datos[vecino])
                   for vecino in vecinos]) / nPuntosG
    dividendo += intra
    #sacamos la minima distancia entre grupos
    centroides_vecinos = np.array(
        [centro for iter,centro  in enumerate(activos)if iter != i] )
    divisor += min(
        [np.linalg.norm(centroide-vecino) for vecino in centroides_vecinos
        ])
  return(dividendo,divisor)

In [5]:
def analizar(mejores,real,datos):
  mejores = resultados(mejores,real,datos)
  mejores = sorted(mejores,key= lambda x: x.aptitud,reverse=True)
  mejorS = mejores[0]; peorS = mejores[-1]; mediaS = mejores[14]
  CSs = [x.cs for x in mejores]
  promedioCS = np.mean(CSs)
  promedioGrupos = np.mean([x.ngrupos for x in mejores])
  pintra =np.mean([x.intra for x in mejores])
  pinter= np.mean([x.inter for x in mejores])
  sortAri = sorted(mejores,key= lambda x: x.ari,reverse=True)
  bari, wari,mari = sortAri[0],sortAri[-1],sortAri[14]
  sortCom = sorted(mejores,key= lambda x: x.completitud,reverse=True)
  bComp, wcomp, mcomp = sortAri[0],sortAri[-1],sortAri[14]
  print(f' datos para analisis estadistico')
  print(f'la mejor solucion tiene f(x): {mejorS.cs}')
  print(f'la peor solucion tiene f(x): {peorS.cs}')
  print(f'mediana soluciones {mediaS.cs}')
  print(f'los resultados tienen una desv estandar de {np.std(CSs)}')
  print(f' datos para comparar con autores')
  print(f'las soluciones tuvieron F(X) promedio de {promedioCS}')
  print(f'las soluciones encontraron en promedio {promedioGrupos} grupos')
  print(f'promedio intra {pintra} promedio inter {pinter}')
  print(f'comparar entre metodos')
  print(f'ari promedio {np.mean([x.ari for x in mejores])}')
  print(f'Mejor ari {bari.ari} peor ari {wari.ari} mediana ari {mari.ari}')
  print(f'DEsviacion ARI {np.std([x.ari for x in sortAri])}')
  print(f'completitud promedio {np.mean([x.completitud for x in mejores])}')
  print(f'Mejor comple {bComp.completitud} peor comple {wcomp.completitud} mediana comple {mcomp.completitud}')
  print(f'DEsviacion Comple {np.std([x.completitud for x in sortCom])}')
  #print(f'en promedio se malclasificaron {np.mean([x.malClasi for x in mejores])} datos')

In [6]:
def resultados(mejoresR,labels,datos):
  mejores = []
  ldatos = len(datos)
  for mejor in mejoresR:
    mejor.ngrupos = len(np.where(mejor.umbrales > 0.5)[0])
    mejor.activos =[x for i,x in enumerate(mejor.centroides)
     if mejor.umbrales[i]>0.5]
    #metricas propuestas para evaluar
    mejor.completitud = completitud(labels,mejor.particion)
    mejor.ari = ari(labels,mejor.particion)
    #####
    errores = confusion(labels,mejor.particion)
    mejor.malClasi = ldatos - sum([errores[i][i] for i in range(len(errores))])
    mejor.cs = (1/mejor.aptitud)-0.0001
    mejor.intra,mejor.inter=D_intra_inter(
        mejor.particion,mejor.activos,datos)
    mejores.append(mejor)
  return(mejores)

In [7]:
def obtener(iterar,*parametros):
  mejores = []
  for iteracion in range(iterar):
    mejores.append(acde(*parametros))
    print(f'terminamos la iteracion {iteracion+1}')
  return(mejores)

In [8]:
def recuperar(archivo):
  with open(archivo, "rb") as f:
      recuperado = pickle.load(f)
  return(recuperado)

# Replicar resultados

## Agrega la ruta a los archivos "replica"

In [9]:
path = '/content' #aqui pon la ruta a la carpeta con los archivos replica

## En el conjunto iris

In [10]:
iris_datos,iris_min,iris_max,iris_labels = recuperar(path+'/replicaIris.pickle')

### usando Cs como aptitud

In [ ]:
iris_cs = obtener(31,1,0.5,10,20,iris_min,iris_max,100000,iris_datos,'cs')

In [ ]:
analizar(iris_cs,iris_real,iris_datos)

### usando DB como aptitud

In [ ]:
iris_db = obtener(31,1,0.5,10,20,iris_min,iris_max,100000,iris_datos,'DB')

In [ ]:
analizar(iris_db,iris_real,iris_datos)

## En el conjunto vino

In [ ]:
vino_datos,vino_min,vino_max,vino_labels = recuperar(path+'/replicaVino.pickle')

### usando Cs como aptitud

In [ ]:
vino_cs = obtener(31,1,0.5,10,20,vino_min,vino_max,100000,vino_datos,'cs')

In [ ]:
analizar(vino_cs,vino_labels,vino_datos)

### usando DB como aptitud

In [ ]:
vino_db = obtener(31,1,0.5,10,20,vino_min,vino_max,100000,vino_datos,'DB')

In [ ]:
analizar(vino_db,vino_labels,vino_datos)

## En el conjunto Vidrio

In [ ]:
vidrio_datos,vidrio_min,vidrio_max,vidrio_labels = recuperar(path+'/replicaVidrio.pickle')

### usando Cs como aptitud

In [ ]:
vidrio_cs = obtener(31,1,0.5,10,20,vidrio_min,vidrio_max,100000,vidrio_datos,'cs')

In [ ]:
analizar(vidrio_cs,vidrio_labels,vidrio_datos)

### usando DB como aptitud

In [ ]:
vidrio_db = obtener(31,1,0.5,10,20,vidrio_min,vidrio_max,100000,vidrio_datos,'DB')

In [ ]:
analizar(vidrio_db,vidrio_labels,vidrio_datos)

## En el conjunto cancer

In [ ]:
cancer_datos,cancer_min,cancer_max,cancer_labels = recuperar(path+'/replicaCancer.pickle')

### usando Cs como aptitud

In [ ]:
cancer_cs = obtener(31,1,0.5,10,20,cancer_min,cancer_max,100000,cancer_datos,'cs')

In [ ]:
analizar(cancer_cs,cancer_labels,cancer_datos)

### usando DB como aptitud

In [ ]:
cancer_db = obtener(31,1,0.5,10,20,cancer_min,cancer_max,100000,cancer_datos,'DB')

In [ ]:
analizar(cancer_db,cancer_labels,cancer_datos)